# DS-610 Week 2 Homework: MapReduce on Apache Spark
This week we will start with a primer on MapReduce and how it can be implemented on Apache Spark.

Our data file is located at: /dbfs/dbfs/FileStore/shared_uploads/dlee5@saintpeters.edu/ds610/pg11.txt

If working locally, make sure to download `pg11.txt` file from the Blackboard website. We will need to change the source file location accordingly below.

This file contains the text *Alice’s Adventures in Wonderland*, by Lewis Carroll. We will run our introductory WordCount example on this text.
The following cell should load the text from the Databricks directory (please do not modify).

**Getting the current Spark Context.** "A Spark Context represents the connection to a Spark cluster, and can be used to create RDDs, accumulators and broadcast variables on that cluster." (from https://learn.microsoft.com/en-us/dotnet/api/microsoft.spark.sparkcontext?view=spark-dotnet) The following code sets up the current Spark Context.

In [0]:
from platform import python_version
print(python_version())
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate()

3.10.12


**Loading the data file.** Let us create an RDD from the file.

In [0]:
# If you are running on Saint Peters' University Databricks Environment, use the following line:
rdd = sc.textFile("dbfs:/FileStore/shared_uploads/dlee5@saintpeters.edu/ds610/pg11.txt")

# If you are running locally, download the dataset from the Blackboard and put it into the same folder as this
# notebook, and uncomment the following line.
#rdd = sc.textFile("pg11.txt")

print('Loaded the data file with %d partitions...' % rdd.getNumPartitions())

Loaded the data file with 2 partitions...


## Part 1
See if we can view the first five rows of the file. We will have to use the *take* method.

https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.take.html

In [0]:
# The solution for Part 1 goes here.
rdd.take(5)

['The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms']

## Part 2
Next we will tokenize each RDD record into a list of words separated by a space " ". Each word also needs to be turned into a lower case. Set the result to the variable called `rdd2`. Then finally take the first five rows from `rdd2'.

For Part 2 we will utilize the `flatMap` function and *lambda* function syntax. For documentation:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.flatMap.html

Note that we are using the `flatMap` instead of `map`.

In [0]:
# The solution for Part 2 goes here.
rdd2 = rdd.flatMap(lambda record: record.lower().split())
first_five_rec = rdd2.take(5)
first_five_rec

['the', 'project', 'gutenberg', 'ebook', 'of']

## Part 3
Apply a filter to exclude tokens that are among the stop word list. Again, the *lambda* function syntax may be useful.

For the stop word list, we may be able to use a manually constructed list such as `['','a','*','and','is','of','the','a']` or one from a package such as WordCloud's stop word list.

The documentation for the function is here:
https://spark.apache.org/docs/3.1.3/api/python/reference/api/pyspark.RDD.filter.html

In [0]:
# The solution for Part 3 goes here.
STOPWORDS = ['','a','*','and','is','of','the','a']

rdd2 = rdd.flatMap(lambda rec: rec.lower().split()).filter(lambda wrd: wrd not in STOPWORDS)
rdd2.take(5)

['project', 'gutenberg', 'ebook', 'alice’s', 'adventures']

## Part 4
Now we need to convert the `rdd2` which is an RDD into a type called a paired RDD. Each RDD is a token word `w` and needs to be mapped to a pair of the form `(w, 1)`. In a paired RDD, the first item in the tuple will serve as the key and the second item as the value. For example, the word `record` will map to `(record, 1)`.

For documentation: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.map.html

Set the resulting paired RDD to a variable called `rdd3`.

In [0]:
# The solution for Part 4 goes here.
rdd3 = rdd2.map(lambda x: (x, 1))
rdd3.take(5)

[('project', 1),
 ('gutenberg', 1),
 ('ebook', 1),
 ('alice’s', 1),
 ('adventures', 1)]

## Part 5
Now let us count how many word tokens we have in our MapReduce job. For documentation:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.count.html

In [0]:
# The solution for Part 5 goes here.
rdd3.count()

25517

## Part 6
Finally let us perform the reduction so that we have `(key, value)` where `key` is a word and `value` is the total number of words. For this we will utilize the following function:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html

Set the result of the reduction to the variable called `rdd4`. Note that `rdd4` is another paired RDD.

In [0]:
# The solution for Part 6 goes here.
from operator import add

rdd4 = rdd3.reduceByKey(add)

rdd4.take(5)

[('project', 83),
 ('gutenberg', 25),
 ('ebook', 8),
 ('adventures', 8),
 ('in', 415)]

## Part 7
Sort the variable `rdd4` by the value in *descending order*. We will need to use the following:
https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.sortBy.html

Afterwards call `collect` method to collect the RDD items into a single Python list onto the driver node. Set the result to the Python variable `sorted_word_count_list`.

In [0]:
# The solution for Part 7 goes here.

sorted_word_count_list = rdd4.sortBy(lambda x: x[1], ascending=False).collect()
print(sorted_word_count_list)

[('to', 784), ('she', 518), ('said', 420), ('in', 415), ('it', 374), ('you', 330), ('was', 330), ('as', 258), ('i', 249), ('that', 230), ('alice', 221), ('with', 217), ('at', 217), ('her', 207), ('had', 176), ('all', 172), ('for', 161), ('be', 157), ('on', 152), ('this', 148), ('or', 143), ('not', 131), ('so', 127), ('very', 127), ('little', 120), ('but', 120), ('“i', 119), ('they', 119), ('he', 110), ('out', 98), ('his', 94), ('if', 93), ('about', 92), ('what', 90), ('by', 83), ('up', 83), ('project', 83), ('were', 82), ('have', 80), ('went', 79), ('one', 79), ('down', 79), ('no', 76), ('alice,', 76), ('like', 75), ('when', 74), ('any', 71), ('would', 71), ('do', 68), ('into', 67), ('there', 65), ('could', 64), ('thought', 63), ('your', 62), ('an', 61), ('its', 60), ('are', 60), ('then', 59), ('who', 58), ('mock', 57), ('my', 56), ('“and', 55), ('gutenberg-tm', 54), ('alice.', 54), ('quite', 53), ('don’t', 52), ('see', 51), ('did', 50), ('some', 50), ('their', 50), ('how', 50), ('them